In [ ]:
%matplotlib inline

import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import math
import pickle
import pandas as pd
import scipy.io
import time
import h5py
import csv


from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

from numpy import linalg as LA
from scipy.spatial import Delaunay
from sklearn.neighbors import NearestNeighbors

#sys.path.insert(0, "../")
from info3d import *
from nn_matchers import *

from query_sets import *

In [ ]:
# Global parameters

radius_range = np.arange(0.5,1.6,0.5)

with open('point_collection/new_contiguous_point_collection.pickle','rb') as f: 
    new_contiguous_point_collection = pickle.load(f)

point_collection_indices = np.arange(len(new_contiguous_point_collection))
point_collection_indices

# Step 0.1: Getting sample points for one-time partial radius

In [ ]:
samples = 1000

sample_points = []

samples_indeces = []

t0 = time.time()

for i in np.arange(samples):

    random_object = np.random.choice(point_collection_indices)

    object_name = new_contiguous_point_collection[random_object][0]
    pointCloud = new_contiguous_point_collection[random_object][1]
    triangles = new_contiguous_point_collection[random_object][2]
    
    triangle_index = np.random.choice(np.arange(len(triangles)))
    vertex_index = triangles[triangle_index,1]
    original_vertex = pointCloud[vertex_index]

    sample_points.append([
        random_object, 
        object_name, 
        original_vertex
    ])
    
    samples_indeces.append(random_object)
    
print("Done generating",len(sample_points),"samples in {:.3f} seconds.".format(time.time()-t0))

with open('sample_points.pickle','wb') as f:
    pickle.dump(sample_points,f)
    
plt.title("Distribution of the sample spaces")
plt.xlabel("Count")
plt.ylabel("Sample space")
plt.hist(samples_indeces,bins = np.arange(0,8))


# Step 0.2: Creating a synthetic set of successive partial spaces

Similar to the partial case above, we use the same sample points, i.e. centroids, for successive releases but will only vary the size of the partial space for every release.


In [ ]:

samples = 100
releases = 100

nearby_range = 2.0
    
t1 = time.time()

successive_sample_points = []

for i in np.arange(samples):#

    random_object = np.random.choice(point_collection_indices)
    #reference_ransac = np.random.randint(5)

    object_name = new_contiguous_point_collection[random_object][0]
    pointCloud = new_contiguous_point_collection[random_object][1]
    triangles = new_contiguous_point_collection[random_object][2]

    current_vertex = pointCloud[np.random.randint(len(pointCloud))]

    growing_point_collection_vertices = [[
        random_object, 
        object_name, 
        current_vertex
    ]]

    nbrs = NearestNeighbors(n_neighbors=min(20000,len(pointCloud)),algorithm='kd_tree').fit(pointCloud[:,:3])

    for release in np.arange(releases-1):

        distances, indices = nbrs.kneighbors([current_vertex[:3]])

        cand_indices = indices[0,np.where(distances[0]<(nearby_range))[0]]

        distribution = np.sort(abs(np.random.normal(nearby_range*0.5,nearby_range*0.3,len(cand_indices))))

        current_vertex = pointCloud[
            np.random.choice(
                cand_indices,
                p = distribution/np.sum(distribution)
            )
        ]

        growing_point_collection_vertices.append([
            random_object, 
            object_name, 
            current_vertex
        ])

    successive_sample_points.append([
        [random_object, object_name],
        growing_point_collection_vertices
    ])

    if i % 33 == 1:
        print(" Done with successive {} sample_points extraction in {:.3f} seconds".format(i,time.time()-t1))
        t1 = time.time()

    with open('successive_sample_points.pickle','wb') as f:
        pickle.dump(successive_sample_points,f)


In [ ]:

t1 = time.time()

try:
    with open('successive_sample_points.pickle','rb') as f:
        successive_point_collection = pickle.load(f)

    samples = len(successive_point_collection)
    releases = len(successive_point_collection[0][1])

    print(samples,"samples for radius",radius)
    print(releases,"releases each")

except Exception as e1:
    print(e1)

successive_sample_points_per_release = [[]]

for k, [obj_, growing_point_collection] in enumerate(successive_point_collection):

    t2 = time.time()

    successive_sample_points = []

    reference_ransac = np.random.randint(5)

    for i, obj_meta in enumerate(growing_point_collection):

        successive_sample_points.append([obj_meta, reference_ransac])

        try:
            successive_sample_points_per_release[i].append(successive_sample_points)
        except:
            successive_sample_points_per_release.append([successive_sample_points])

        #print(len(successive_sample_points_per_release[i]),len(successive_sample_points_per_release[i][k]))

    with open('successive_sample_points_per_release.pickle','wb') as f:
        pickle.dump(successive_sample_points_per_release,f)

print(" Done with successive sample_points extraction in {:.3f} seconds".format(time.time()-t1))
    

# Step 0.3: Create submaps for pointnetvlad using same samples

In [ ]:
spatial_span = 2.0

interval = 0.5

num_points = 4096

cutoff = 0.5

with open('sample_points.pickle','rb') as f:
    sample_points = pickle.load(f)

# Step 0.3.1: Generate the reference dataset using the raw dataset

In [ ]:
baseline_path = 'pointnetvlad_submaps/'

if not os.path.exists(baseline_path): os.mkdir(baseline_path)

raw_path = os.path.join(baseline_path,"raw_dataset")
raw_pc_path = os.path.join(raw_path,"pointcloud_4m_0.25")

if not os.path.exists(raw_path): os.mkdir(raw_path)
if not os.path.exists(raw_pc_path): os.mkdir(raw_pc_path)

t0 = time.time()

csvfile = open(raw_path+"/pointcloud_centroids_4m_0.25.csv",'w',newline = '')

csv_writer = csv.writer(csvfile, delimiter = ',')
csv_writer.writerow(['timestamp', 'northing', 'easting','alting','obj'])

for obj_, [object_name, pointCloud, triangles] in enumerate(new_contiguous_point_collection):

    if object_name == "Reception-Data61-L5.obj":
        new_X = pointCloud[:,0] - np.mean(pointCloud[:,0]) + 50
        new_Z = pointCloud[:,2] - np.mean(pointCloud[:,2]) + 0
    elif object_name == "Driveway.obj":
        new_X = pointCloud[:,0] - np.mean(pointCloud[:,0]) - 25
        new_Z = pointCloud[:,2] - np.mean(pointCloud[:,2]) - 50
    elif object_name == "Apartment.obj":
        new_X = pointCloud[:,0] - np.mean(pointCloud[:,0]) + 25
        new_Z = pointCloud[:,2] - np.mean(pointCloud[:,2]) - 50
    elif object_name == "Workstations-Data61-L4.obj":
        new_X = pointCloud[:,0] - np.mean(pointCloud[:,0]) - 50
        new_Z = pointCloud[:,2] - np.mean(pointCloud[:,2]) + 0
    elif object_name == "Kitchen-Data61-L4.obj":
        new_X = pointCloud[:,0] - np.mean(pointCloud[:,0]) + 0
        new_Z = pointCloud[:,2] - np.mean(pointCloud[:,2]) + 0
    elif object_name == "HallWayToKitchen-Data61-L4.obj":
        new_X = pointCloud[:,0] - np.mean(pointCloud[:,0]) - 25
        new_Z = pointCloud[:,2] - np.mean(pointCloud[:,2]) + 50
    elif object_name == "StairWell-Data61-L4.obj":
        new_X = pointCloud[:,0] - np.mean(pointCloud[:,0]) + 25
        new_Z = pointCloud[:,2] - np.mean(pointCloud[:,2]) + 50

    new_Y = pointCloud[:,1]
        
    new_object_pointcloud = np.stack((new_X,new_Z,new_Y)).T

    nbrs = NearestNeighbors(n_neighbors=20000, algorithm='kd_tree').fit(new_object_pointcloud)

    round_new_pointcloud = 0.25*100*np.around((0.01/0.25)*new_object_pointcloud,decimals=2)
    unq_round_pointcloud = np.unique(round_new_pointcloud[:,:3],axis = 0)

    raw_centroids = unq_round_pointcloud#+np.random.normal(0,0.25,unq_round_pointcloud.shape)

    for northing, easting, alting in raw_centroids:

        # Getting the points around our centroid defined by [northing, easting]
        distances, indices = nbrs.kneighbors([[northing, easting, alting]])

        #if max(distances[0]) < 0.5*spatial_span: continue

        submap_pointcloud = new_object_pointcloud[indices[0,np.where(distances[0,:]<=spatial_span)[0]]]

        if len(submap_pointcloud) == 0:
            continue

        # Centering and rescaling
        submap_pointcloud = (submap_pointcloud - [northing, easting, alting])/spatial_span

        if len(submap_pointcloud) > num_points:
            submap_pointcloud = submap_pointcloud[np.random.choice(len(submap_pointcloud),num_points)]
        elif len(submap_pointcloud) < num_points and len(submap_pointcloud) >= cutoff*num_points :
            #print(i,submap_pointcloud.shape)
            additional_pointcloud = submap_pointcloud[np.random.choice(len(submap_pointcloud),num_points-len(submap_pointcloud))]
            additional_pointcloud = additional_pointcloud + np.random.normal(0,0.05,additional_pointcloud.shape)
            submap_pointcloud = np.concatenate((submap_pointcloud,additional_pointcloud),axis = 0)
        elif len(submap_pointcloud) < cutoff*num_points :
            #continue
            #print(i,submap_pointcloud.shape)
            additional_pointcloud = submap_pointcloud[np.random.choice(len(submap_pointcloud),num_points-len(submap_pointcloud), True)]
            additional_pointcloud = additional_pointcloud + np.random.normal(0,0.05,additional_pointcloud.shape)
            submap_pointcloud = np.concatenate((submap_pointcloud,additional_pointcloud),axis = 0)

        timestamp = int(10**16*(time.time()))

        csv_writer.writerow([timestamp,northing,easting,alting,obj_])

        with open(raw_pc_path+'/{}.pickle'.format(timestamp),'wb') as f:
            pickle.dump(submap_pointcloud.T,f)

    print("Done with submap generation for object ({}) {} in {:.3f} seconds".format(obj_,object_name,time.time()-t0))

csvfile.close()


# Step 0.3.2: Generate a reference dataset using a sample RANSAC dataset

In [ ]:
# First, we need to create the reference dataset using the raw dataset and a (randomly chosen) ransac dataset.

num_points = 4096

baseline_path = 'pointnetvlad_submaps/'

if not os.path.exists(baseline_path): os.mkdir(baseline_path)
    
try:
    trial = np.random.randint(5)

    with open("../ransac_pc/ransac_point_collection_{}.pickle".format(trial),'rb') as f:
        ransac_trial_point_collection = pickle.load(f)

    object_, pointCloud_, tri_ = ransac_trial_point_collection[0]
    
    print("Chosen ransac trial",trial)
    
except Exception as ex:
    print("Error:",ex)

raw_path = os.path.join(baseline_path,"ransac_dataset")
raw_pc_path = os.path.join(raw_path,"pointcloud_4m_0.25")

if not os.path.exists(raw_path): os.mkdir(raw_path)
if not os.path.exists(raw_pc_path): os.mkdir(raw_pc_path)

t0 = time.time()

csvfile = open(raw_path+"/pointcloud_centroids_4m_0.25.csv",'w',newline = '')

csv_writer = csv.writer(csvfile, delimiter = ',')
csv_writer.writerow(['timestamp', 'northing', 'easting','alting','obj'])

for obj_, [object_name, pointCloud, triangles] in enumerate(ransac_trial_point_collection):

    if object_name == "Reception-Data61-L5.obj":
        new_X = pointCloud[:,0] - np.mean(pointCloud[:,0]) + 50
        new_Z = pointCloud[:,2] - np.mean(pointCloud[:,2]) + 0
    elif object_name == "Driveway.obj":
        new_X = pointCloud[:,0] - np.mean(pointCloud[:,0]) - 25
        new_Z = pointCloud[:,2] - np.mean(pointCloud[:,2]) - 50
    elif object_name == "Apartment.obj":
        new_X = pointCloud[:,0] - np.mean(pointCloud[:,0]) + 25
        new_Z = pointCloud[:,2] - np.mean(pointCloud[:,2]) - 50
    elif object_name == "Workstations-Data61-L4.obj":
        new_X = pointCloud[:,0] - np.mean(pointCloud[:,0]) - 50
        new_Z = pointCloud[:,2] - np.mean(pointCloud[:,2]) + 0
    elif object_name == "Kitchen-Data61-L4.obj":
        new_X = pointCloud[:,0] - np.mean(pointCloud[:,0]) + 0
        new_Z = pointCloud[:,2] - np.mean(pointCloud[:,2]) + 0
    elif object_name == "HallWayToKitchen-Data61-L4.obj":
        new_X = pointCloud[:,0] - np.mean(pointCloud[:,0]) - 25
        new_Z = pointCloud[:,2] - np.mean(pointCloud[:,2]) + 50
    elif object_name == "StairWell-Data61-L4.obj":
        new_X = pointCloud[:,0] - np.mean(pointCloud[:,0]) + 25
        new_Z = pointCloud[:,2] - np.mean(pointCloud[:,2]) + 50

    new_Y = pointCloud[:,1]
        
    new_object_pointcloud = np.stack((new_X,new_Z,new_Y)).T

    nbrs = NearestNeighbors(n_neighbors=20000, algorithm='kd_tree').fit(new_object_pointcloud)

    round_new_pointcloud = 0.25*100*np.around((0.01/0.25)*new_object_pointcloud,decimals=2)
    unq_round_pointcloud = np.unique(round_new_pointcloud[:,:3],axis = 0)

    raw_centroids = unq_round_pointcloud#+np.random.normal(0,0.25,unq_round_pointcloud.shape)

    for northing, easting, alting in raw_centroids:

        # Getting the points around our centroid defined by [northing, easting]
        distances, indices = nbrs.kneighbors([[northing, easting, alting]])

        #if max(distances[0]) < 0.5*spatial_span: continue

        submap_pointcloud = new_object_pointcloud[indices[0,np.where(distances[0,:]<=spatial_span)[0]]]

        if len(submap_pointcloud) == 0:
            continue

        # Centering and rescaling
        submap_pointcloud = (submap_pointcloud - [northing, easting, alting])/spatial_span

        if len(submap_pointcloud) > num_points:
            submap_pointcloud = submap_pointcloud[np.random.choice(len(submap_pointcloud),num_points)]
        elif len(submap_pointcloud) < num_points and len(submap_pointcloud) >= cutoff*num_points :
            #print(i,submap_pointcloud.shape)
            additional_pointcloud = submap_pointcloud[np.random.choice(len(submap_pointcloud),num_points-len(submap_pointcloud))]
            additional_pointcloud = additional_pointcloud + np.random.normal(0,0.05,additional_pointcloud.shape)
            submap_pointcloud = np.concatenate((submap_pointcloud,additional_pointcloud),axis = 0)
        elif len(submap_pointcloud) < cutoff*num_points :
            #continue
            #print(i,submap_pointcloud.shape)
            additional_pointcloud = submap_pointcloud[np.random.choice(len(submap_pointcloud),num_points-len(submap_pointcloud), True)]
            additional_pointcloud = additional_pointcloud + np.random.normal(0,0.05,additional_pointcloud.shape)
            submap_pointcloud = np.concatenate((submap_pointcloud,additional_pointcloud),axis = 0)

        timestamp = int(10**16*(time.time()))


        csv_writer.writerow([timestamp,northing,easting,alting,obj_])

        with open(raw_pc_path+'/{}.pickle'.format(timestamp),'wb') as f:
            pickle.dump(submap_pointcloud.T,f)

    print("Done with submap generation for object ({}) {} in {:.3f} seconds".format(obj_,object_name,time.time()-t0))

csvfile.close()

# Step 0.3.3: Generate the test submaps:
 - using Raw spaces for validation
 - using Ransac spaces for evaluation
 - using Ransac spaces for the successive case

In [ ]:
# One-time releases Raw partial spaces

baseline_path = 'pointnetvlad_submaps/'

if not os.path.exists(baseline_path): os.mkdir(baseline_path)

for radius in np.arange(0.25,3.1,0.25):
    
    per_radius_partial_length = []
    
    t1 = time.time()
            
    partial_path = os.path.join(baseline_path,"raw_partial_radius_"+str(radius)+"_"+str(num_points))+"_unassisted"
    pointcloud_partial_path = os.path.join(partial_path,"pointcloud_4m")
    #pointcloud_partial_bin_path = os.path.join(partial_path,"pointcloud_4m_npy")

    if not os.path.exists(partial_path): os.mkdir(partial_path)
    if not os.path.exists(pointcloud_partial_path): os.mkdir(pointcloud_partial_path)
    #if not os.path.exists(pointcloud_partial_bin_path): os.mkdir(pointcloud_partial_bin_path)

    print(" ",pointcloud_partial_path)
    #"""
    csvfile = open(partial_path+"/pointcloud_centroids_4m.csv",'w',newline = '')

    csv_writer = csv.writer(csvfile, delimiter = ',')
    csv_writer.writerow(['timestamp', 'northing', 'easting', 'alting','obj'])
    #"""
    count = 0
    
    for obj_, object_name, original_vertex in sample_points:
        
        new_partial_pointcloud = []
        new_vX = []
        new_vZ = []
        
        try:

            object_, ransac_pointCloud, tri_ = new_contiguous_point_collection[int(obj_)]
            
            ransac_nbrs = NearestNeighbors(n_neighbors=min(20000,len(ransac_pointCloud)), algorithm='kd_tree').fit(ransac_pointCloud[:,:3])
            
            dist_, ind_ = ransac_nbrs.kneighbors([original_vertex[:3]])
            pointCloud =  ransac_pointCloud[ind_[0,np.where(dist_[0,:]<=radius)[0]]]
        except:
            print("Can't get ransac samples for",trial,obj_meta[0],dist_.shape,ind_.shape)
            continue
            
        #if len(gen_planes) == 0: continue
        if len(pointCloud) == 0: continue

        if object_name == "Reception-Data61-L5.obj":
            new_X = pointCloud[:,0] + 50
            new_Z = pointCloud[:,2] + 0
            new_vX = original_vertex[0] + 50
            new_vZ = original_vertex[2] + 0
        elif object_name == "Driveway.obj":
            new_X = pointCloud[:,0] - 25
            new_Z = pointCloud[:,2] - 50
            new_vX = original_vertex[0] - 25
            new_vZ = original_vertex[2] - 50
        elif object_name == "Apartment.obj":
            new_X = pointCloud[:,0] + 25
            new_Z = pointCloud[:,2] - 50
            new_vX = original_vertex[0] + 25
            new_vZ = original_vertex[2] - 50
        elif object_name == "Workstations-Data61-L4.obj":
            new_X = pointCloud[:,0] - 50
            new_Z = pointCloud[:,2] + 0
            new_vX = original_vertex[0] - 50
            new_vZ = original_vertex[2] + 0
        elif object_name == "Kitchen-Data61-L4.obj":
            new_X = pointCloud[:,0] + 0
            new_Z = pointCloud[:,2] + 0
            new_vX = original_vertex[0] + 0
            new_vZ = original_vertex[2] + 0
        elif object_name == "HallWayToKitchen-Data61-L4.obj":
            new_X = pointCloud[:,0] - 25
            new_Z = pointCloud[:,2] + 50
            new_vX = original_vertex[0] - 25
            new_vZ = original_vertex[2] + 50
        elif object_name == "StairWell-Data61-L4.obj":
            new_X = pointCloud[:,0] + 25
            new_Z = pointCloud[:,2] + 50
            new_vX = original_vertex[0] + 25
            new_vZ = original_vertex[2] + 50
        else:
            print("Error:",obj_meta)

        new_Y = pointCloud[:,1]

        new_partial_pointcloud = np.stack((new_X,new_Z,new_Y)).T
        
        max_known_span = max(np.amax(new_partial_pointcloud, axis = 0) - np.amin(new_partial_pointcloud, axis = 0))
        
        nbrs = NearestNeighbors(n_neighbors=min(2*num_points,len(new_partial_pointcloud)), algorithm='kd_tree').fit(new_partial_pointcloud)

        # Get submap "centroids" by quantizing by 0.25m, i.e. round then unique
        if max_known_span > 3*spatial_span:
            round_new_partial_pointcloud = 100*np.around(0.01*new_partial_pointcloud,decimals=2)
            unq_round_partial_pointcloud = np.unique(round_new_partial_pointcloud[:,:3],axis = 0)
        #    
            raw_partial_centroids = unq_round_partial_pointcloud
            c_nbrs = NearestNeighbors(n_neighbors = min(25,len(raw_partial_centroids)),  algorithm='kd_tree').fit(raw_partial_centroids)
            c_dist, c_ind = c_nbrs.kneighbors(raw_partial_centroids)

            ia1, ia2 = np.where(c_dist < 1.73)
            

            dist_bins = np.bincount(ia1)
            max_dist = max(np.bincount(ia1))
            raw_partial_centroids = raw_partial_centroids[[i for i, j in enumerate(dist_bins) if j == max_dist]]
            
            raw_partial_centroids = raw_partial_centroids+np.random.normal(0,interval,raw_partial_centroids.shape)
            
        else:
            # Correcting this, because the attacker is supposed to not know the true centroid
            # and has to estimate it instead.
            #raw_partial_centroids = [[new_vX, new_vZ, original_vertex[1]]]        

            raw_partial_centroids = [np.mean(new_partial_pointcloud, axis = 0)]
            
        for northing, easting, alting in raw_partial_centroids:
            
            # Getting the points around our centroid defined by [northing, easting]
            distances, indices = nbrs.kneighbors([[northing, easting, alting]])
            
            #if max(distances[0]) < 0.5*spatial_span: continue
                
            submap_pointcloud = new_partial_pointcloud[indices[0,np.where(distances[0,:]<=spatial_span)[0]]]

            if len(submap_pointcloud) == 0:
                continue
            
            # Centering and rescaling
            submap_pointcloud = (submap_pointcloud - [northing, easting, alting])/spatial_span

            if len(submap_pointcloud) > num_points:
                submap_pointcloud = submap_pointcloud[np.random.choice(len(submap_pointcloud),num_points)]
            elif len(submap_pointcloud) < num_points and len(submap_pointcloud) >= cutoff*num_points :
                #print(i,submap_pointcloud.shape)
                additional_pointcloud = submap_pointcloud[np.random.choice(len(submap_pointcloud),num_points-len(submap_pointcloud))]
                additional_pointcloud = additional_pointcloud + np.random.normal(0,0.05,additional_pointcloud.shape)
                submap_pointcloud = np.concatenate((submap_pointcloud,additional_pointcloud),axis = 0)
            elif len(submap_pointcloud) < cutoff*num_points :
                #continue
                #print(i,submap_pointcloud.shape)
                additional_pointcloud = submap_pointcloud[np.random.choice(len(submap_pointcloud),num_points-len(submap_pointcloud), True)]
                additional_pointcloud = additional_pointcloud + np.random.normal(0,0.05,additional_pointcloud.shape)
                submap_pointcloud = np.concatenate((submap_pointcloud,additional_pointcloud),axis = 0)

            timestamp = int(10**16*(time.time()))
            
            csv_writer.writerow([timestamp,northing,easting,alting,obj_])
            
            with open(pointcloud_partial_path+'/{}.pickle'.format(timestamp),'wb') as f:
                pickle.dump(submap_pointcloud.T,f)

            count += 1


    print("   Done with submap generation for radius {} ( {} samples) in {:.3f} seconds".format(radius,count,time.time()-t1))
    csvfile.close()
       

In [ ]:
# One-time releases RANSAC partial spaces

baseline_path = 'pointnetvlad_submaps/'

if not os.path.exists(baseline_path): os.mkdir(baseline_path)

for radius in np.arange(0.25,3.1,0.25):
        
    t1 = time.time()
            
    partial_path = os.path.join(baseline_path,"ransac_partial_radius_"+str(radius)+"_"+str(num_points))+"_unassisted"
    pointcloud_partial_path = os.path.join(partial_path,"pointcloud_4m")
    #pointcloud_partial_bin_path = os.path.join(partial_path,"pointcloud_4m_npy")

    if not os.path.exists(partial_path): os.mkdir(partial_path)
    if not os.path.exists(pointcloud_partial_path): os.mkdir(pointcloud_partial_path)
    #if not os.path.exists(pointcloud_partial_bin_path): os.mkdir(pointcloud_partial_bin_path)

    print(" ",pointcloud_partial_path)
    #"""
    csvfile = open(partial_path+"/pointcloud_centroids_4m.csv",'w',newline = '')

    csv_writer = csv.writer(csvfile, delimiter = ',')
    csv_writer.writerow(['timestamp', 'northing', 'easting', 'alting','obj'])
    #"""
    count = 0
    
    for obj_, object_name, original_vertex in sample_points:
        
        new_partial_pointcloud = []
        new_vX = []
        new_vZ = []
        
        try:
            trial = np.random.randint(5)
            
            with open("../ransac_pc/ransac_point_collection_{}.pickle".format(trial),'rb') as f:
                ransac_trial_point_collection = pickle.load(f)

            object_, ransac_pointCloud, tri_ = ransac_trial_point_collection[int(obj_)]
            
            ransac_nbrs = NearestNeighbors(n_neighbors=min(20000,len(ransac_pointCloud)), algorithm='kd_tree').fit(ransac_pointCloud[:,:3])
            
            dist_, ind_ = ransac_nbrs.kneighbors([original_vertex[:3]])
            pointCloud =  ransac_pointCloud[ind_[0,np.where(dist_[0,:]<=radius)[0]]]
        except:
            print("Can't get ransac samples for",trial,obj_meta[0])
            continue
            
        #if len(gen_planes) == 0: continue
        if len(pointCloud) == 0: continue

        if object_name == "Reception-Data61-L5.obj":
            new_X = pointCloud[:,0] + 50
            new_Z = pointCloud[:,2] + 0
            new_vX = original_vertex[0] + 50
            new_vZ = original_vertex[2] + 0
        elif object_name == "Driveway.obj":
            new_X = pointCloud[:,0] - 25
            new_Z = pointCloud[:,2] - 50
            new_vX = original_vertex[0] - 25
            new_vZ = original_vertex[2] - 50
        elif object_name == "Apartment.obj":
            new_X = pointCloud[:,0] + 25
            new_Z = pointCloud[:,2] - 50
            new_vX = original_vertex[0] + 25
            new_vZ = original_vertex[2] - 50
        elif object_name == "Workstations-Data61-L4.obj":
            new_X = pointCloud[:,0] - 50
            new_Z = pointCloud[:,2] + 0
            new_vX = original_vertex[0] - 50
            new_vZ = original_vertex[2] + 0
        elif object_name == "Kitchen-Data61-L4.obj":
            new_X = pointCloud[:,0] + 0
            new_Z = pointCloud[:,2] + 0
            new_vX = original_vertex[0] + 0
            new_vZ = original_vertex[2] + 0
        elif object_name == "HallWayToKitchen-Data61-L4.obj":
            new_X = pointCloud[:,0] - 25
            new_Z = pointCloud[:,2] + 50
            new_vX = original_vertex[0] - 25
            new_vZ = original_vertex[2] + 50
        elif object_name == "StairWell-Data61-L4.obj":
            new_X = pointCloud[:,0] + 25
            new_Z = pointCloud[:,2] + 50
            new_vX = original_vertex[0] + 25
            new_vZ = original_vertex[2] + 50
        else:
            print("Error:",obj_meta)

        new_Y = pointCloud[:,1]

        new_partial_pointcloud = np.stack((new_X,new_Z,new_Y)).T
        
        max_known_span = max(np.amax(new_partial_pointcloud, axis = 0) - np.amin(new_partial_pointcloud, axis = 0))
        
        nbrs = NearestNeighbors(n_neighbors=min(2*num_points,len(new_partial_pointcloud)), algorithm='kd_tree').fit(new_partial_pointcloud)

        # Get submap "centroids" by quantizing by 0.25m, i.e. round then unique
        if max_known_span > 3*spatial_span:
            round_new_partial_pointcloud = 100*np.around(0.01*new_partial_pointcloud,decimals=2)
            unq_round_partial_pointcloud = np.unique(round_new_partial_pointcloud[:,:3],axis = 0)
        #    
            raw_partial_centroids = unq_round_partial_pointcloud
            c_nbrs = NearestNeighbors(n_neighbors = min(25,len(raw_partial_centroids)),  algorithm='kd_tree').fit(raw_partial_centroids)
            c_dist, c_ind = c_nbrs.kneighbors(raw_partial_centroids)

            ia1, ia2 = np.where(c_dist < 1.73)
            

            dist_bins = np.bincount(ia1)
            max_dist = max(np.bincount(ia1))
            raw_partial_centroids = raw_partial_centroids[[i for i, j in enumerate(dist_bins) if j == max_dist]]
            
            raw_partial_centroids = raw_partial_centroids+np.random.normal(0,interval,raw_partial_centroids.shape)
            
        else:
            # Correcting this, because the attacker is supposed to not know the true centroid
            # and has to estimate it instead.
            #raw_partial_centroids = [[new_vX, new_vZ, original_vertex[1]]]        

            raw_partial_centroids = [np.mean(new_partial_pointcloud, axis = 0)]
            
        for northing, easting, alting in raw_partial_centroids:
            
            # Getting the points around our centroid defined by [northing, easting]
            distances, indices = nbrs.kneighbors([[northing, easting, alting]])
            
            #if max(distances[0]) < 0.5*spatial_span: continue
                
            submap_pointcloud = new_partial_pointcloud[indices[0,np.where(distances[0,:]<=spatial_span)[0]]]

            if len(submap_pointcloud) == 0:
                continue
            
            # Centering and rescaling
            submap_pointcloud = (submap_pointcloud - [northing, easting, alting])/spatial_span

            if len(submap_pointcloud) > num_points:
                submap_pointcloud = submap_pointcloud[np.random.choice(len(submap_pointcloud),num_points)]
            elif len(submap_pointcloud) < num_points and len(submap_pointcloud) >= cutoff*num_points :
                #print(i,submap_pointcloud.shape)
                additional_pointcloud = submap_pointcloud[np.random.choice(len(submap_pointcloud),num_points-len(submap_pointcloud))]
                additional_pointcloud = additional_pointcloud + np.random.normal(0,0.05,additional_pointcloud.shape)
                submap_pointcloud = np.concatenate((submap_pointcloud,additional_pointcloud),axis = 0)
            elif len(submap_pointcloud) < cutoff*num_points :
                #continue
                #print(i,submap_pointcloud.shape)
                additional_pointcloud = submap_pointcloud[np.random.choice(len(submap_pointcloud),num_points-len(submap_pointcloud), True)]
                additional_pointcloud = additional_pointcloud + np.random.normal(0,0.05,additional_pointcloud.shape)
                submap_pointcloud = np.concatenate((submap_pointcloud,additional_pointcloud),axis = 0)

            timestamp = int(10**16*(time.time()))
            

            csv_writer.writerow([timestamp,northing,easting,alting,obj_])
            
            with open(pointcloud_partial_path+'/{}.pickle'.format(timestamp),'wb') as f:
                pickle.dump(submap_pointcloud.T,f)

            count += 1

    print("   Done with submap generation for radius {} ( {} samples) in {:.3f} seconds".format(radius,count,time.time()-t1))
    csvfile.close()
       

In [ ]:
# Successive release of RANSAC partial spaces

baseline_path = 'pointnetvlad_submaps/'

skip = 5

t0 = time.time()

with open('successive_sample_points_per_release.pickle','rb') as f:
    successive_sample_points_per_release = pickle.load(f)

for radius in np.arange(0.5,2.1,0.5):
    
    t1 = time.time()

    successive_path = os.path.join(baseline_path,"successive_radius_"+str(radius))    
    
    if not os.path.exists(successive_path): os.mkdir(successive_path)
    
    for i in np.arange(1,100,skip): # releases
        
        successive_release_path = os.path.join(successive_path,"release_"+str(i))
        pointcloud_successive_path = os.path.join(successive_release_path,"pointcloud_4m")
        
        if not os.path.exists(successive_release_path): os.mkdir(successive_release_path)
        if not os.path.exists(pointcloud_successive_path): os.mkdir(pointcloud_successive_path)
                                    
        csvfile = open(successive_release_path+"/pointcloud_centroids_4m.csv",'w',newline = '')

        csv_writer = csv.writer(csvfile, delimiter = ',')
        csv_writer.writerow(['timestamp', 'northing', 'easting', 'alting','obj'])
        
        count = 0
        
        for successive_sample_points_per_release_per_obj in successive_sample_points_per_release[i]:
    
            #print("  ",len(successive_sample_points_per_release_per_obj),"releases")
            growing_point_cloud = []
            
            new_vX = []
            new_vZ = []
            
            ransac_pointCloud = []
                
            for [obj_, object_name, original_vertex], reference_ransac in successive_sample_points_per_release_per_obj[:i]:

                try:
                    if len(ransac_pointCloud) == 0: # if empty, open. This only happens at beginning
                        with open("../ransac_pc/ransac_point_collection_{}.pickle".format(reference_ransac),'rb') as f:
                            ransac_trial_point_collection = pickle.load(f)

                        object_, ransac_pointCloud, tri_ = ransac_trial_point_collection[int(obj_)]
                        
                        ransac_nbrs = NearestNeighbors(n_neighbors=min(20000,len(ransac_pointCloud)), algorithm='kd_tree').fit(ransac_pointCloud[:,:3])

                except:
                    print("Can't get ransac samples for",i,obj_, object_name)
                    continue

                dist_, ind_ = ransac_nbrs.kneighbors([original_vertex[:3]])
                pointCloud =  ransac_pointCloud[ind_[0,np.where(dist_[0,:]<=radius)[0]]]

                if len(pointCloud) == 0: continue
                    
                #Regular Accumulation
                if len(growing_point_cloud) == 0:
                    growing_point_cloud = pointCloud

                else:
                    growing_point_cloud = np.concatenate(
                        (growing_point_cloud,pointCloud),
                        axis=0
                    )
                    
            if len(growing_point_cloud) == 0: continue
                    
            pointCloud = np.unique(growing_point_cloud,axis=0)     

            if object_name == "Reception-Data61-L5.obj":
                new_X = pointCloud[:,0] + 50
                new_Z = pointCloud[:,2] + 0
                new_vX = original_vertex[0] + 50
                new_vZ = original_vertex[2] + 0
            elif object_name == "Driveway.obj":
                new_X = pointCloud[:,0] - 25
                new_Z = pointCloud[:,2] - 50
                new_vX = original_vertex[0] - 25
                new_vZ = original_vertex[2] - 50
            elif object_name == "Apartment.obj":
                new_X = pointCloud[:,0] + 25
                new_Z = pointCloud[:,2] - 50
                new_vX = original_vertex[0] + 25
                new_vZ = original_vertex[2] - 50
            elif object_name == "Workstations-Data61-L4.obj":
                new_X = pointCloud[:,0] - 50
                new_Z = pointCloud[:,2] + 0
                new_vX = original_vertex[0] - 50
                new_vZ = original_vertex[2] + 0
            elif object_name == "Kitchen-Data61-L4.obj":
                new_X = pointCloud[:,0] + 0
                new_Z = pointCloud[:,2] + 0
                new_vX = original_vertex[0] + 0
                new_vZ = original_vertex[2] + 0
            elif object_name == "HallWayToKitchen-Data61-L4.obj":
                new_X = pointCloud[:,0] - 25
                new_Z = pointCloud[:,2] + 50
                new_vX = original_vertex[0] - 25
                new_vZ = original_vertex[2] + 50
            elif object_name == "StairWell-Data61-L4.obj":
                new_X = pointCloud[:,0] + 25
                new_Z = pointCloud[:,2] + 50
                new_vX = original_vertex[0] + 25
                new_vZ = original_vertex[2] + 50
            else:
                print("Error:",[obj_, object_name, original_vertex])

            new_Y = pointCloud[:,1]

            new_partial_pointcloud = np.stack((new_X,new_Z,new_Y)).T

            nbrs = NearestNeighbors(n_neighbors=min(2*num_points,len(new_partial_pointcloud)), algorithm='kd_tree').fit(new_partial_pointcloud)

            # Get submap "centroids" by quantizing by 0.25m, i.e. round then unique
            #if radius > spatial_span:
            round_new_partial_pointcloud = 100*np.around(0.01*new_partial_pointcloud,decimals=2)
            unq_round_partial_pointcloud = np.unique(round_new_partial_pointcloud[:,:3],axis = 0)
        #    
            partial_centroids = unq_round_partial_pointcloud+np.random.normal(0,interval,unq_round_partial_pointcloud.shape)
            c_nbrs = NearestNeighbors(n_neighbors = min(25,len(partial_centroids)),  algorithm='kd_tree').fit(partial_centroids)
            c_dist, c_ind = c_nbrs.kneighbors(partial_centroids)

            ia1, ia2 = np.where(c_dist < 1.73)

            dist_bins = np.bincount(ia1)
            max_dist = max(np.bincount(ia1))
            partial_centroids = partial_centroids[[i for i, j in enumerate(dist_bins) if j == max_dist]]

            for northing, easting, alting in partial_centroids:

                # Getting the points around our centroid defined by [northing, easting]
                distances, indices = nbrs.kneighbors([[northing, easting, alting]])

                #if max(distances[0]) < 0.5*spatial_span: continue

                submap_pointcloud = new_partial_pointcloud[indices[0,np.where(distances[0,:]<=spatial_span)[0]]]

                if len(submap_pointcloud) == 0:
                    continue

                # Centering and rescaling
                submap_pointcloud = (submap_pointcloud - [northing, easting, alting])/spatial_span
                #per_radius_partial_length.append([northing, easting, alting, len(submap_pointcloud)])

                if len(submap_pointcloud) > num_points:
                    submap_pointcloud = submap_pointcloud[np.random.choice(len(submap_pointcloud),num_points)]
                elif len(submap_pointcloud) < num_points and len(submap_pointcloud) >= cutoff*num_points :
                    #print(i,submap_pointcloud.shape)
                    additional_pointcloud = submap_pointcloud[np.random.choice(len(submap_pointcloud),num_points-len(submap_pointcloud))]
                    additional_pointcloud = additional_pointcloud + np.random.normal(0,0.05,additional_pointcloud.shape)
                    submap_pointcloud = np.concatenate((submap_pointcloud,additional_pointcloud),axis = 0)
                elif len(submap_pointcloud) < cutoff*num_points :
                    #continue
                    #print(i,submap_pointcloud.shape)
                    additional_pointcloud = submap_pointcloud[np.random.choice(len(submap_pointcloud),num_points-len(submap_pointcloud), True)]
                    additional_pointcloud = additional_pointcloud + np.random.normal(0,0.05,additional_pointcloud.shape)
                    submap_pointcloud = np.concatenate((submap_pointcloud,additional_pointcloud),axis = 0)

                timestamp = int(10**16*(time.time()))

                csv_writer.writerow([timestamp,northing,easting,alting,obj_])

                with open(pointcloud_successive_path+'/{}.pickle'.format(timestamp),'wb') as f:
                    pickle.dump(submap_pointcloud.T,f)
                    
                count += 1
                    
        if i % 10 == 1:
            print("   Done with submap generation for iteration {}, radius {} ({} submaps) in {:.3f} seconds".format(i,radius,count,time.time()-t1))
            t1 = time.time()

        csvfile.close()
        
    print(" Done with generalized submap generation for radius {} in {:.3f} seconds".format(radius,time.time()-t0))
    t0 = time.time()
                

# Step 0.3.4: Building database and query files for evaluation with pointnetVLAD 
 - the combined Raw and RANSAC referece database 
 - for validation with one-time released Raw spaces
 - for testing with one-time released RANSAC spaces
 - for testing with successive RANSAC spaces

In [ ]:
base_path= "pointnetvlad_submaps/"#"../partial_dataset/"

construct_query_and_database_sets(
    base_path, 
    ['raw_dataset', 'ransac_dataset'], 
    "/pointcloud_4m_0.25/",
    "pointcloud_centroids_4m_0.25.csv")#, all_folders[index])

In [ ]:
# For validation with raw queries.

for radius in np.arange(0.25,3.1,0.25):
    
    partial_path = 'raw_partial_radius_'+str(radius)+"_4096_unassisted"#
    
    print(partial_path)
    construct_query_sets(partial_path, "/pointcloud_4m/", "pointcloud_centroids_4m.csv")#, all_folders[index])


In [ ]:
# For the Ransac queries.

for radius in np.arange(0.25,3.1,0.25):
    
    partial_path = 'ransac_partial_radius_'+str(radius)+"_4096_unassisted"#
    
    print(partial_path)
    construct_query_sets(partial_path, "/pointcloud_4m/", "pointcloud_centroids_4m.csv")#, all_folders[index])


In [ ]:
# For the successive queries.

successive_dir = os.path.join(base_path,"successive_queries")    

if not os.path.exists(successive_dir): os.mkdir(successive_dir)

for radius in np.arange(0.5,2.1,0.5):
    
    successive_path = 'successive_radius_'+str(radius)
    
    for release in np.arange(1,100,5):
    
        partial_path = 'release_'+str(release)

        print(partial_path)
        construct_successive_query_sets(successive_path,partial_path, "/pointcloud_4m/", "pointcloud_centroids_4m.csv")#, all_folders[index])

#print(all_folders)
#print("training:",train_folders)
#	